In [3]:
class ARGS():
    def __init__(self):
        self.gpus = '0'
        self.cpus = 4
        self.seed = 42
        self.network = 'GEM'
        self.n_tasks = 2
        self.n_memories = 256
        self.memory_strength = 0.5
        self.model = 'ResNet18'
        self.channel = 64
        self.load = False
        self.official = False
        self.checkpoint = 'checkpoints'
        self.epochs = 5
        self.loss = 'CE'
        self.init_lr = 1e-2
        self.lr_scheduler = 'cosine_annealing'
        self.optimizer = 'Adam'
        self.eval_term = 1
        self.dataset = 'mnist'
        self.continuum = True
        self.data_path = 'data'
        self.image_size = [256, 256]
        self.batch_size = 10
        self.in_channel = 3
args = ARGS()

In [11]:
from network.utils import load_network
import torch

device = 'cpu'
network, start_epoch, best_recall, recalls, save_path = load_network(args, 10, device=device)

Create ResNet18 model


In [12]:
network.grad_dims[:10]

[9408, 64, 64, 64, 36864, 64, 64, 64, 36864, 64]

In [13]:
network.grads.shape

torch.Size([11186442, 2])

In [25]:
network.grads[:, 0].shape

torch.Size([11186442])

In [26]:
network.grads[:, 0].unsqueeze(0).shape

torch.Size([1, 11186442])

In [29]:
network.grads.index_select(1, torch.LongTensor([0])).shape

torch.Size([11186442, 1])

In [30]:
torch.mm(network.grads[:, 0].unsqueeze(0),
         network.grads.index_select(1, torch.LongTensor([0])))

tensor([[0.]])

In [31]:
network.grads[:, 0].unsqueeze(1).shape

torch.Size([11186442, 1])